### **4.11 - How the Webserver works?**

En este vídeo, vamos a ver cómo funciona el webserver en Airflow. Si recuerdas el video anterior, mencioné el parámetro 'worker_refresh_interval' en el archivo de configuración de Airflow. Este parámetro permite establecer el número de segundos a esperar antes de refrescar un lote de workers (batch of workers).

La pregunta es, ¿qué es un worker?

Una cosa que hay que saber es que cuando se inicia el webserver desde la interfaz de línea de comandos, en realidad se inicia un servidor Gunicorn utilizando el framework web de python Flask. Gunicorn es un servidor HTTP de Python para UNIX basado en el modelo pre-fork worker. Este modelo significa que hay un proceso maestro central que gestiona un conjunto de procesos fork worker. El proceso maestro es un bucle que escucha varias señales de proceso y reacciona en consecuencia para gestionar una lista de workers en ejecución. Estos workers son responsables de analizar los DAGs en la carpeta dags, así como de gestionar las peticiones y devolver una respuesta al cliente. Es por eso que si vas al archivo airflow.cfg, en la sección webserver, tienes parámetros como 'web_server_master_timeout' y 'web_ server_worker_timeout'. Te preguntarás, ¿en qué caso el master o los workers pueden hacer timeout? Bueno, ya que el webserver se ejecuta localmente está limitado por los recursos en términos de CPU y ancho de banda de la red. Sus peticiones no deberían tardar un tiempo imprevisible en terminar ya que no estamos solicitando a través de Internet. Recuerde, estoy hablando de los workers que manejan las solicitudes y respuestas del webserver de Airflow, no de los workers que ejecutan las tareas de sus DAGs. Así que, volviendo a los timeouts, si un worker tarda demasiado tiempo en dar una respuesta, es destruido y reiniciado. Por defecto, los timeouts están configurados a 2 minutos. Puedes modificar esos valores si empiezas a tener tus workers destruidos debido a los timeouts de los logs del webserver, pero los valores por defecto deberían ser suficientes. Hablando de los logs del webserver, vamos a ir a la terminal e iniciar el contenedor que ejecuta Airflow con el script start.sh. Una vez iniciado, escribimos "docker ps", luego "docker logs -f", copiamos y pegamos el id del contenedor y pulsamos enter. Muy bien, ahora podemos ver los logs vamos a desplazarnos un poco hacia arriba hasta llegar al mensaje que dice "Running the Gunicorn Server With".

<center><img src="https://i.postimg.cc/CKSjfv6F/a708.png"></center>

Aquí mismo. Justo debajo de esta línea, podemos ver que tenemos 4 workers sincrónicos. La palabra "sync" aquí significa sincrónico. Entonces, ¿por qué tenemos 4 workers y por qué son sincrónicos? Comencemos con los tipos de worker. Gunicorn ofrece múltiples tipos de worker pero nos vamos a centrar sólo en dos de ellos que son los workers síncronos y asíncronos. Los worker sincrónicos son el tipo por defecto de Gunicorn. En este modelo, cada worker manejará una y sólo una petición a la vez. Este tipo de worker se utiliza con aplicaciones que no necesitan un disco largo de I/O o peticiones externas. ¿Por qué? Porque bloquearán las demás peticiones, lo que las hará fallar debido a los timeouts. Por otro lado, los worker asíncronos son más adecuados para manejar más de una petición que no son predecibles por naturaleza. De acuerdo, ya sabemos por qué Airflow se basa en workers síncronos, pero ¿por qué sólo 4? Se trata de la escalabilidad. Básicamente, se podría decir que cuantos más workers pongamos, más peticiones podrá atender el webserver. Aunque eso no es falso, no deberías hacerlo. En algún momento, si añades demasiados workers, empezarás a desperdiciar recursos del sistema disminuyendo el rendimiento de tu máquina. Ten en cuenta que Gunicorn sólo debería necesitar entre cuatro y doce procesos worker para poder gestionar miles de peticiones por segundo. Ya puede ver que es mucho más alto que nuestras necesidades, ya que el webserver de Airflow va a ser utilizado por pocas personas. Como mejor práctica, generalmente se recomienda utilizar la fórmula (2 * el número de cores) + 1 para establecer el número de workers. En nuestro caso, nos quedaremos con el mínimo de 4, que es más que suficiente para Airflow. Si volvemos al archivo airflow.cfg, unas pocas líneas por debajo de los timeouts, podemos encontrar los parámetros 'workers' y 'worker_class' respectivamente establecidos en 4 y sync. Muy bien, ahora hay dos parámetros más que quiero mostrar que son el 'worker_refresh_batch_size' y 'worker_refresh_interval'. 'worker_refresh_batch_size' define el número de workers a refrescar a la vez. Cuando se establece en 0, la actualización de los workers está desactivada, cuando se establece en un valor distinto de cero, airflow refresca periódicamente los workers del webserver trayendo nuevos y destruyendo los antiguos. Por otro lado, el parámetro 'worker_refresh_interval' define el tiempo a esperar antes de hacer ese refresco. Te preguntarás por qué se produce este refresco. Bueno, volvamos al terminal antes de darte la respuesta. En los logs, justo debajo de la línea con equals (=====), podemos ver el proceso maestro de gunicorn con el pid 27 en mi caso, starting the gunicorn server, listening at localhost:8080 y así sucesivamente. Luego tenemos cuatro líneas que muestran que los workers están arrancando (Booting). ¿Por qué cuatro? Porque el parámetro 'workers' está puesto a cuatro.

<center><img src="https://i.postimg.cc/52qz1B9D/a709.png"></center>

Luego, si te desplazas hacia abajo, verás las líneas de log "Handling signal" con la señal "ttin" o "ttou". Básicamente, esas señales se envían al proceso maestro para aumentar o disminuir el número de workers. Eso es exactamente lo que ocurre aquí. Cuando se inicia un nuevo worker, el más antiguo se termina. Si miras cuidadosamente el tiempo aquí, este proceso de refresco ocurre cada 30 segundos como se define en el parámetro 'worker_refresh_interval' y sólo un worker es refrescado en cada intervalo. Bien, eso es genial, pero todavía no he respondido por qué Airflow necesita refrescar esos workers. Bueno, ¿recuerdas los timeout parameters? Esa es la razón por la que los workers se refrescan. Si un worker se agota por cualquier razón, Airflow iniciará uno nuevo con el fin de mantener el mismo número de workers en ejecución como se define en airflow.cfg. Por defecto, cada 'worker_refresh_interval', el webserver crea un nuevo worker y como el número de workers se hace mayor que el número de workers definido en el parámetro workers, el worker más antiguo se detiene.

<center><img src="https://i.postimg.cc/YqFyHC4T/a710.png"></center>

Muy bien, lo último de lo que quiero hablar es del nivel de logs que puedes definir en el parámetro 'logging_level'. Vayamos al editor de código, y busquemos el parámetro 'logging_level'. Por defecto, está establecido en INFO que es el nivel más alto de logs. Un nivel que puede ser muy útil es DEBUG. Como se puede adivinar por el nombre, permite obtener mucho más logs con el fin de depurar o entender lo que está pasando cuando Airflow se está ejecutando. No voy a hacerlo ahora, pero siéntase libre de cambiar el nivel y comprobar por sí mismo la diferencia. Como mejor práctica, nunca debe utilizar este nivel en la producción. Sólo en desarrollo.

<center><img src="https://i.postimg.cc/BncYj7Bg/a711.png"></center>